<a href="https://colab.research.google.com/github/neurologic/NeurophysiologyModules/blob/main/EMG_AntagonisticPairs_Interossei_CodingLight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Contents
1. [Introduction](#scrollTo=850eb426-07d8-4407-82f1-f163a2c960b6)
2. [Initialize Notebook](#scrollTo=GsbjCNcAw9VA)
3. [Raw EMG Signal](#scrollTo=0c80eb80-1d17-47bb-81a8-023bf0c2476d)
4. [Select Movement](#scrollTo=jE6GzivbzjVP)
5. [Calculate EMG envelope](#scrollTo=0d1f1545-2f25-4bfe-be34-1dfb7ca3444e)
6. [Analyze Muscle Coordination](#scrollTo=84f7e282-b954-42bd-8ea2-b5edb134ec99)
7. [Connect Muscles to Movement](#scrollTo=0d494468-1f12-4f71-9196-9a77e1633a38)

<hr>

## 1. Introduction
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

By thoughtfully working through the analysis in this notebook, you will learn more about what it means for neural/muscle activity to be coordinated, and how activity in the motor system relates to movement in the world. 
Some of the terms and analytic techniques that you will be using:
- EMG
- amplitude envelope
- correlation
- kinematics

Throughout the notebook, you will plot both raw and processed data.
- You can interact with the plots by zooming in and panning. <br>
- You can save the current plot view at any time by hitting the "download" icon - it will save to your Downloads folder. Make sure to re-name the auto-generated file and make notes about what you plotted right away. <br>

To aquire this dataset, two differential emg electrodes were used to simultaneously record emg activity at 100Hz from two dorsal interossei muscles - one on either side of the second digit (at the axial line). A video camera simultaneously [captured images at 30 frames per second](https://drive.google.com/file/d/1pcTWfYmSyoXg-aSzfjAr7cAzLZnyGQTd/view?usp=sharing).

<img src='https://drive.google.com/uc?export=view&id=170GkCR_9lii5930iedjNcdhb36YigyYM' width=300px>

There were 5 different movement conditions to explore how movement of the second digit relates to electrical activity in its lateral antagonists. 
<br>
<br>

--- 

### **Assignment**: 
After going through the notebook from start to finish and exploring the data, use steps 6 and 7 to make two figures that compare two finger movement conditions of your choice. 
 - Figure 1: Muscle Coordination. 
 - Figure 2: The relationship between movement and muscle.
 
Include a concise legend and a results/discussion paragraph for each figure . 

---





## 2. Initialize notebook.
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))


In [ ]:
#@markdown **TASK:** Run this code cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@markdown **TASK:** Run this code cell to load packages, 
#@markdown initialize the notebook, and load the dataset you will explore and analyze.
#@markdown > **NOTE:** If you are loading these files from a different 
#@markdown location than the BIOL358 course, change the filepaths as needed below
#@markdown before running.
filepath_emg = '/content/drive/Shareddrives/BIOL358/Data/Interossei_2chan_WithVideo/emg_Resampled_2022-01-10T11_39_23.csv' #@param {type:"string"}
filepath_joints = '/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/EMG_FingerAntagonists_MuscleShieldWithVideo/video2022-01-10T11_39_23DLC_resnet50_FingerMovementJan10shuffle1_200000_filtered.h5' #@param {type:"string"}
filepath_bones = '/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/EMG_FingerAntagonists_MuscleShieldWithVideo/video2022-01-10T11_39_23DLC_resnet50_FingerMovementJan10shuffle1_200000_skeleton.h5' #@param {type:"string"}
filepath_video_timestamps = '/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/EMG_FingerAntagonists_MuscleShieldWithVideo/vid_timestamps2022-01-10T11_39_23.csv' #@param {type:"string"}
'''
This notebook assumes that you have access to the shared
data on Google Drive.
Alternatively, you can upload a file to session storage 
and change this filepath.
'''


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import csv
from scipy.signal import hilbert,medfilt,resample
from sklearn.decomposition import PCA
import scipy
import seaborn as sns
from datetime import datetime,timezone,timedelta
pal = sns.color_palette(n_colors=15)
pal = pal.as_hex()

fs = 1000
fs_vid = 30


# datafolder = "/content/drive/Shareddrives/BIOL358/Data/Interossei_2chan_WithVideo"
# emg_file = "emg_Resampled_2022-01-10T11_39_23.csv"
# emg_path = Path(datafolder) / emg_file
emg_path = Path(filepath_emg)

# import data
df_analog = pd.read_csv(emg_path.expanduser().resolve(), sep=',').drop(columns=['Unnamed: 0'])

def get_bodypart_speed(df_vid,d,new_name):
    dd = np.diff(df_vid[d].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = speed #(speed - np.mean(speed))/np.std(speed)
    return df_vid

def get_bone_speed(df_vid,bone,new_name):
    dd = np.diff(df_vid[bone].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = speed #(speed - np.mean(speed))/np.std(speed)
    return df_vid

# datafolder = '/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/EMG_FingerAntagonists_MuscleShieldWithVideo'
# joints_file = 'video2022-01-10T11_39_23DLC_resnet50_FingerMovementJan10shuffle1_200000_filtered.h5'
# bones_file = 'video2022-01-10T11_39_23DLC_resnet50_FingerMovementJan10shuffle1_200000_skeleton.h5'
# videotimestamps_file = "vid_timestamps2022-01-10T11_39_23.csv"

joints_path = Path(filepath_joints) #Path(datafolder) / joints_file
bones_path = Path(filepath_bones) #Path(datafolder) / bones_file
video_path = Path(filepath_video_timestamps) #Path(datafolder) / videotimestamps_file

bodyparts = ['2_Tip','2_K1','2_K2','2_K3','1_K1']
bones = ['2_Tip_2_K3']

## Now import video data and do some processing
##########
offset_vidstart = 1.0742 #offset time between video start and analog recording start (video starts first)

df_vid = pd.read_hdf(joints_path) # import data from body parts _filtered.h5
df_vid = df_vid[df_vid.columns[0][0]]
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['_x','_y']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]

# filter by bodyparts
bodyparts = [any(keep in ele for keep in bodyparts) for ele in list(df_vid.columns)]
df_vid_bodyparts = df_vid.loc[:,(np.asarray(coords) & np.asarray(bodyparts))]

df_vid = pd.read_hdf(bones_path) # import data from bones _skeleton.h5
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['orientation']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]
# filter by bodyparts
bones = [any(keep in ele for keep in bones) for ele in list(df_vid.columns)]
df_vid_bones = df_vid.loc[:,(np.asarray(coords) & np.asarray(bones))]

df_vid = df_vid_bones.join(df_vid_bodyparts) # combine bones and bodyparts
df_vid['time'] = (df_vid.index.values / fs_vid) - offset_vidstart # create time column
# df_vid = df_vid[((df_vid['time']>(start)) & (df_vid['time']<(stop)))] # select start:stop section

df_vid = get_bodypart_speed(df_vid,'2_Tip_y','2_Tip_y_speed')
df_vid = get_bodypart_speed(df_vid,'2_Tip_x','2_Tip_x_speed')
df_vid = get_bone_speed(df_vid,'2_Tip_2_K3_orientation','2_Tip_2_K3_speed')

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))


###### NOTE TO INSTRUCTORS COLLECTING RAW DATA #######
# If have not processed the data yet: need to do this. Save as a "pre-processing step" for this lab
# emg_file_preprocessed = "emg_channels2022-01-10T11_39_23.csv"
# emg_path = Path(folderpath) / emg_file_preprocessed
# #%% import data
# df_analog_ = pd.read_csv(emg_path.expanduser().resolve(), sep=',')
# # reformat 'Time' to start at t0 = 0 and be units of seconds
# # xtime = (df_analog['Time']-df_analog['Time'].iloc[0])/1000
# # reformat 'Time' to be units of seconds
# xtime = (df_analog_['time'])/1000
# df_analog_['time'] = xtime
# x = np.linspace(df_analog_['time'].iloc[0],df_analog_['time'].iloc[-1],
#                 int((df_analog_['time'].iloc[-1]-df_analog_['time'].iloc[0])*fs))
# df_mat = [df_analog_.loc[np.max(df_analog_[df_analog_['time']<=x_].index)].values for x_ in x]
# df_analog = pd.DataFrame(df_mat,columns = df_analog_.columns)
# # df_analog = df_analog.loc[:,channels]
# df_analog['time']=x-x[0]
## pick a new file name
# df_analog.to_csv(Path(folderpath) / 'resampled' + emg_file_preprocessed)

## 3. Plot the raw EMG signal from each interossei muscle
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

> The amplitude units of an EMG signal are in Volts. For this lab, we will focus on relative EMG amplitude, not its absolute value. In the next step, you will be obtaining the "amplitude envelope" of this raw signal, which discards absolute Voltage information. <br>
<br>
> The "baseline" of each channels is offset from each other so that they are not visually overlaid - the baseline voltage of the recording on each channel was actually zero. 


In [ ]:
#@markdown **TASK:** Run this cell to plot the raw EMG data.
#@markdown You can then zoom in and scroll around to explore the data. 

fig = go.Figure()
fig.add_trace(go.Scatter(x = df_analog['time'].values, y = df_analog['emg0'].values,line_color='orange',name='lateral'))
fig.add_trace(go.Scatter(x = df_analog['time'].values, y = df_analog['emg1'].values + 300,line_color='purple',name='medial'))
fig.update_layout(xaxis_title="time(seconds)", 
                  yaxis_title='amplitude',
                  width=800, height=500)
fig.layout.yaxis.showticklabels=False
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
fig.show()

## 4. Analyze a selection of the data. 
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

In [ ]:
#@markdown Pick a time window to analyze that corresponds to 
#@markdown which movement you want to analyze 
#@markdown and enter it in the code cell below.
#@markdown > Note: get the time by hovering over the data plot. 
#@markdown - **start** = the start time of the bout of repeated movement (seconds).
#@markdown - **stop** = the stop time of the bout of repeated movement (seconds).

start = None #@param {type:"number"}
stop = None #@param {type:"number"}

#@markdown <b>Task:</b> After you have specified the start and stop times,
#@markdown run this cell to execute the variable assignment.
df_vid_selection = df_vid[((df_vid['time']>(start)) & (df_vid['time']<(stop)))] # select start:stop section
df_analog_selection = df_analog[((df_analog['time']>start) & (df_analog['time']<stop))]

print('all set - analysis domain defined')
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## 5. Process the raw EMG signal into a rectified, smoothed "amplitude envelope"
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

> This is a type of *signal processing*. You have already encountered this type of signal processing when you analyzed the extracellular cockroach data that you collected earlier this semester. Remind yourself of what this does to the raw signal that you recorded. The result here is a processed signal that we will refer to as the *amplitude envelope* of the EMG activity. 


In [ ]:
#@markdown <b>Task:</b> Run this cell to compute the envelope of the 
#@markdown raw EMG signal for the specified time window
#@markdown and plot of the result. 

# Use rectfication and gaussian smoothing on EMG to get mean-centered rate
df_rate = pd.DataFrame({})
filtert = int(0.05*fs)
for h in list(df_analog_selection.drop(columns='time').columns): # rename headers as input channels
    y = df_analog_selection[h] - np.mean(df_analog_selection[h])
    y = np.abs(y) #takes the absolute value of 
    y = scipy.ndimage.gaussian_filter(y,filtert)
    df_rate[h] = y
# df_rate = df_rate.subtract(df_rate.mean())
# df_rate =(df_rate - df_rate.mean()) / df_rate.std()
df_rate['time']=df_analog_selection['time'].values
# df_rate = df_rate[((df_rate['time']>start) & (df_rate['time']<stop))]


fig = go.Figure()
fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg0'].values,line_color='orange',name='lateral'))
fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg1'].values,line_color='purple',name='medial'))
fig.update_layout(xaxis_title="time(seconds)", 
                  yaxis_title='amplitude',width=800, height=400)
fig.layout.xaxis.showticklabels=False
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
fig.show()

## 6. Muscle coordination
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

Think about how you would qualitatively describe the relationship between the two muscle signals that you plotted. The correlation metric is often helpful to quantify the relationship between signals. Essentially, correlation is the measure of how two or more variables are related to one another. https://en.wikipedia.org/wiki/Correlation



In [ ]:
#@markdown <b>Task:</b> Run this cell to: <br> 1. calculate the 
#@markdown mathematical correlation values between the two signals and
#@markdown <br> 2. plot the lateral and medial EMG envelopes against each other. 
#@markdown - (Think about how the scatterplot is a way to visualize *correlation*.
#@markdown Does the scatterplot change the way you interpret the correlation value that you calculated?)

# No need to edit this code cell
################################
print('correlation matrix:')
p_corr = df_rate.drop('time',axis=1).corr()
# hfig, ax = plt.subplots(1)
# sns.heatmap(p_corr, annot=True)
display(p_corr)

fig = go.Figure()
fig.add_trace(go.Scatter(x = df_rate['emg0'].values, 
                         y = df_rate['emg1'].values,
                        #  marker_color='black',
                         mode = 'markers', 
                         marker=dict(
                          color='black',
                          size=2,
                          opacity=0.5,)))
fig.update_layout(xaxis_title="amplitude lateral interossei", 
                  yaxis_title='amplitude medial interossei',
                  width=400, height=400)
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
# axs.set_ylabel('amplitude (a.u.) of your second EMG channel')
# axs.set_xlabel('amplitude (a.u.) of your first EMG channel');


## 7. Connecting movements and muscles. 
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

Now let's examine how motor system activity relates to movement kinematics. 

Individual landmarks on the hand were tracked using DeepLabCut. You can [view the video with tracked landmarks labelled](https://drive.google.com/file/d/17ML1ReBxOSoizQ2JaUkg1jiJj6nGiyDB/view?usp=sharing) (including "trailing" points of the past 5 positions of each landmark). The angle of speficic joint connections ('bones') were also calculated from the raw tracking data using DeepLabCut. In this section, you can explore the kinematics and how they relate to the muscle activity. 

In [ ]:
#@markdown Activate Dropdown Menus.
#@markdown >Note: Once activated, you do not need to re-run this 
#@markdown code cell to change your selection. 

# print('')
# print('sample of table with positions of body parts and orientations of bones')
# display(df_vid_selection.drop('time',axis=1))

w_kinematics = widgets.SelectMultiple(
    options=list(df_vid_selection.drop('time',axis=1).keys()),
    rows=10,
    disabled=False
)
print('')
print('')
print('body parts and bones that you can now analyze for kinematics:')

display(w_kinematics)
print('Dropdown generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

Can you tell which of the kinematic variables are raw and which are 'processed'? 

**TASK:** Select kinematic data from the list above that you want to visually compare to muscle activity. You can change your selection at any time after activating the menu (no need to re-activate if you are able to edit your selections).


In [ ]:
#@markdown **TASK:** Run this code cell to plot the emg envelopes 
#@markdown and the selected kinematic variables. 
#@markdown > Note: the y-axis label on the kinematics plot
#@markdown is blank because different variables will have different units. 
#@markdown You can add your own manually later in another program.

fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)

fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg0'].values,
                         line_color='orange',name='lateral'),
              row=1, col=1)
fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg1'].values,
                         line_color='purple',name='medial'),
              row=1, col=1)

if len(w_kinematics.value)>0:
    for k in list(w_kinematics.value):
        fig.add_trace(go.Scatter(x = df_vid_selection['time'].values, 
                                 y = df_vid_selection[k].values,name=k),
                      row=2, col=1)
    print('Plot created at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if len(w_kinematics.value)==0:
    print('Need to select markers to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

fig.update_layout(xaxis2_title="time(seconds)", yaxis_title='emg amplitude',width=800, height=500)
fig.show()

# fig = make_subplots(rows=4, cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing=0.02)

# fig.add_trace(go.Scatter(x = df_vid['time'].values, y = df_vid['2_Tip_2_K3_speed'].values,line_color=sns.xkcd_rgb['black'],name='2_Tip_2_K3_speed'),
#               row=3, col=1)

# fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg0'].values,line_color='orange',name='emg0'),
#               row=4, col=1)
# fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg1'].values,line_color='purple',name='emg1'),
#               row=4, col=1)

# fig.add_trace(go.Scatter(x = df_vid['time'].values, y = df_vid['2_K2_x'].values,line_color=sns.xkcd_rgb['green'],name='2_K2_x'),
#               row=1, col=1)
# fig.add_trace(go.Scatter(x = df_vid['time'].values, y = df_vid['2_K2_y'].values,line_color=sns.xkcd_rgb['aqua'],name='2_K2_y'),
#               row=1, col=1)

# fig.add_trace(go.Scatter(x = df_vid['time'].values, y = df_vid['2_Tip_2_K3_orientation'].values,line_color=sns.xkcd_rgb['red'],name='2_Tip_2_K3_orientation'),
#               row=2, col=1)


# fig.update_layout(width=1000, height=800)

# fig = go.Figure()
# for k,c in zip(kinematics_to_plot,colors_to_plot):
#   fig.add_trace(go.Scatter(x = df_vid['time'].values, y = df_vid[k].values,line_color=sns.xkcd_rgb[c],name=k))
# fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg0'].values,line_color='orange',name='emg0'))
# fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg1'].values,line_color='purple',name='emg1'))
# fig.update_layout(xaxis_title="time(seconds)", yaxis_title='amplitude',width=1000, height=500)
# fig.show()
# hfig,ax = plt.subplots(1)
# for k,c in zip(kinematics_to_plot,colors_to_plot):
#     ax.plot(df_vid['time'],df_vid[k],linewidth = 2,color = sns.xkcd_rgb[c],label = k,alpha = 0.75)
# ax.plot(df_rate['time'],df_rate['emg0'].values,linewidth = 2,color = sns.xkcd_rgb['blue'],alpha = 0.75,label = 'EMG channel 0')
# ax.plot(df_rate['time'],df_rate['emg1'].values,linewidth = 2,color = sns.xkcd_rgb['orange'],alpha = 0.75,label = 'EMG channel 1')
# plt.legend();

Think about how you would qualitatively describe the relationship between the two muscle signals and the kinematic variables (and the relationship among kinematic variables). Come up with some hypotheses to quantitatively test. What metrics would you use to test your hypotheses?